*Copyright (C) 2021 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

# Hierarchical _Processes_ and _SubProcessModels_

Previous tutorials have briefly covered that there are two categories of _ProcessModels_: _LeafProcessModels_ and _SubProcessModels_. The [ProcessModel Tutorial](#tutorial03_process_models.ipynb) explained _LeafProcessModels_ in detail. These implement the behavior of a _Process_ directly, in the language (for example, Python or Loihi Neurocore API) required for a particular compute resource (for example, a CPU or Loihi Neurocores). _SubProcessModels_, by contrast, allow users to implement and compose the behavior of a process _using other processes_. This enables the creation of Hierarchical _Processes_ and reuse of primitive _Process_ Models to realize more complex _ProcessModels_. _SubProcessModels_ inherit all compute resource requirements from the sub _Processes_ they instantiate. 

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial07/fig01_subprocessmodels.png"/>

In this tutorial, we will create a Dense Layer Hierarchical Process that has the behavior of a fully connected layer of Leaky-Integrate-and-Fire (LIF) neurons. The Dense Layer Process Model implements this behavior via the primitive LIF and Dense Connection Processes and their respective PyLoihiProcessModels.

## Recommended tutorials before starting: 

- [Installing Lava](./tutorial01_installing_lava.ipynb "Tutorial on Installing Lava")
- [Processes](./tutorial02_processes.ipynb "Tutorial on Processes")
- [ProcessModel](./tutorial03_process_models.ipynb "Tutorial on ProcessModels")
- [Execution](./tutorial04_execution.ipynb "Tutorial on Executing Processes")
- [Connecting Processes](./tutorial05_connect_processes.ipynb "Tutorial on Connecting Processes")

## Create a Dense Connection _Process_ and _ProcessModel_

The [ProcessModel Tutorial](#tutorial03_process_models.ipynb) walks through the creation of a LIF _Process_ and an implementing _PyLoihiProcessModel_. Our DenseLayer _Process_ also requires a Dense Lava _Process_ and _ProcessModel_ that have the behavior of a dense set of synaptic connections and weights. The Dense Connection _Process_ can be used to connect neural _Processes_. For completeness, we'll first show how to create this Dense Connection _Process_ and _ProcessModel_.

#### Create a Dense Connection Process

In [15]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort


class Dense(AbstractProcess):
    """Dense connections between neurons.
    Realizes the following abstract behavior:
    a_out = W * s_in
    """

    def __init__(self, **kwargs):
        # super(AbstractProcess, self).__init__(kwargs)
        # shape = kwargs.pop("shape")
        super().__init__(**kwargs)
        shape = kwargs.get("shape", (1, 1))
        self.s_in = InPort(shape=(shape[1],))
        self.a_out = OutPort(shape=(shape[0],))
        self.weights = Var(shape=shape, init=kwargs.pop("weights", 0))

#### Create a Python Dense Connection Process Model implementing the Loihi Sync Protocol and requiring a CPU compute resource

In [16]:
import numpy as np

from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.resources import CPU
from lava.magma.core.decorator import implements, requires
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.proc.dense.process import Dense


@implements(proc=Dense, protocol=LoihiProtocol)
@requires(CPU)
class PyDenseModel(PyLoihiProcessModel):
    s_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, bool, precision=1)
    a_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, np.int32, precision=16)
    weights: np.ndarray = LavaPyType(np.ndarray, np.int32, precision=8)

    def run_spk(self):
        s_in = self.s_in.recv()
        a_out = self.weights[:, s_in].sum(axis=1)
        self.a_out.send(a_out)

## Create a DenseLayer Hierarchical _Process_ that encompasses Dense and LIF _Process_ behavior

Now we create a Dense Layer Hierarchical Process combing LIF neural Processes and Dense connection Processes. Our Hierarchical Process contains all of the variables (`u`, `v`, `bias`, `du`, `dv`, and `vth`) native to the LIF Process plus the `weights` variable native to the Dense Process. The InPort to our Hierarchical Process is `s_in`, which represents the input spikes to our Dense synaptic connections. These Dense connections synapse onto a population of LIF neurons. The OutPort of our Hierarchical Process is `s_out`, which represents the spikes output by the layer of LIF neurons.

In [17]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort

class DenseLayer(AbstractProcess):
    """Combines Dense and LIF Processes.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get("shape", (1, 1))
        self.s_in = InPort(shape=(shape[1],))
        #output of Dense synaptic connections is only used internally
        #self.a_out = OutPort(shape=(shape[0],))
        self.weights = Var(shape=shape, init=kwargs.pop('weights', 0))
        #input to LIF population from Dense synaptic connections is only used internally
        #self.a_in = InPort(shape=(shape[0],))
        self.s_out = OutPort(shape=(shape[0],))
        self.u = Var(shape=(shape[0],), init=0)
        self.v = Var(shape=(shape[0],), init=0)
        self.bias = Var(shape=(shape[0],), init=kwargs.pop("bias", 0))
        self.du = Var(shape=(1,), init=kwargs.pop("du", 0))
        self.dv = Var(shape=(1,), init=kwargs.pop("dv", 0))
        self.vth = Var(shape=(1,), init=kwargs.pop("vth", 10))

## Create a _SubProcessModel_ that implements the DenseLayer _Process_ using Dense and LIF child _Processes_

Now we will create the _SubProcessModel_ that implements our DenseLayer _Process_. This inherits from the _AbstractSubProcessModel_ class. Recall that _SubProcessModels_ also inherit the compute resource requirements from the child _Processes_ they instantiate. Therefore, _SubDenseLayerModel_ implicitly requires the CPU compute resource required by our LIF and Dense child _Processes_. 

The ports (`s_in`,`s_out`) and variables (`u`,`v`,`bias`,`du`,`dv`,`vth`,`weights`) of the _SubProcessModel_ `SubDenseLayerModel` must exactly match (in name and number) the ports and variables of the `DenseLayer` _Process_ for proper compilation. As is true for all _PyProcessModels_, the ports of `SubDenseLayerModel` have type _PyInPort_ and _PyOutPort_, and the variables of `SubDenseLayerModel` must be initialized as the _LavaPyType_.

The `__init__()` method of `SubDenseLayerModel` builds the sub _Process_ structure of the `DenseLayer` _Process_. The `DenseLayer` _Process_ gets passed to the `__init__()` method via the `proc` attribute. The `__init__()` method first instantiates the child LIF and Dense _Processes_. Initial conditions of the `DenseLayer` _Process_, which are required to instantiate the child LIF and Dense _Processes_, are accessed through `proc.init_args`. 

We then `connect()` the in-port of the Dense child _Process_ to the in-port of the `DenseLayer` parent _Process_ and the out-port of the LIF child _Process_ to the out-port of the `DenseLayer` parent _Process_. Note that ports of the `DenseLayer` parent process are accessed using `proc.in_ports` or `proc.out_ports`, while ports of a child _Process_ like LIF are accessed using `self.lif.in_ports` and `self.lif.out_ports`. Our _ProcessModel_ also internally `connect()`s the out-port of the Dense connection child _Process_ to the in-port of the LIF neural child _Process_. 

The `alias()` method exposes the variables of the LIF and Dense child _Processes_ to the `DenseLayer` parent _Process_. Note that the variables of the `DenseLayer` parent _Process_ are accessed using `proc.vars`, while the variables of a child `Process` like LIF are accessed using `self.lif.vars`.

In [18]:
import numpy as np

from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
from lava.magma.core.model.sub.model import AbstractSubProcessModel

from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.resources import CPU
from lava.magma.core.decorator import implements, requires

@implements(proc=DenseLayer, protocol=LoihiProtocol)
class SubDenseLayerModel(AbstractSubProcessModel):
    """Implements Process behavior via sub Processes."""
    s_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, bool, precision=1)
    s_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, bool, precision=1)
    
    u: np.ndarray = LavaPyType(np.ndarray, np.int32, precision=24)
    v: np.ndarray = LavaPyType(np.ndarray, np.int32, precision=24)
    bias: np.ndarray = LavaPyType(np.ndarray, np.int16, precision=12)
    du: int = LavaPyType(int, np.uint16, precision=12)
    dv: int = LavaPyType(int, np.uint16, precision=12)
    vth: int = LavaPyType(int, int, precision=8)
    weights: np.ndarray = LavaPyType(np.ndarray, np.int32, precision=8)

    def __init__(self, proc):
        """Builds sub Process structure of the Process."""
        # Instantiate child processes
        #input shape is a 2D vec (shape of weight mat)
        shape = proc.init_args.get("shape",(1,1))
        weights = proc.init_args.get("weights",(1,1))
        bias = proc.init_args.get("bias",(1,1))
        vth = proc.init_args.get("vth",(1,1))
        #shape is a 2D vec (shape of weight mat)
        self.dense = Dense(shape=shape, weights=weights)
        #shape is a 1D vec
        self.lif = LIF(shape=(shape[0],),b=bias,vth=vth)
        # connect Parent in port to child Dense in port
        proc.in_ports.s_in.connect(self.dense.in_ports.s_in)
        # connect Dense Proc out port to LIF Proc in port
        self.dense.out_ports.a_out.connect(self.lif.in_ports.a_in)
        # connect child LIF out port to parent out port
        self.lif.out_ports.s_out.connect(proc.out_ports.s_out)

        proc.vars.u.alias(self.lif.vars.u)
        proc.vars.v.alias(self.lif.vars.v)
        proc.vars.bias.alias(self.lif.vars.bias)
        proc.vars.du.alias(self.lif.vars.du)
        proc.vars.dv.alias(self.lif.vars.dv)
        proc.vars.vth.alias(self.lif.vars.vth)
        proc.vars.weights.alias(self.dense.vars.weights)

## Run the DenseLayer _Process_

#### Compile 1 DenseLayer _Process_

In [20]:
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

#test compilation

#randomly initialize weights
weights = np.random.randint(5,size=(2,6))
#instantiate a DenseLayer
denseLayer = DenseLayer(shape=(2,6),weights=weights, bias=3)
#retrieve the weights from the DenseLayer
print('weights: \n',denseLayer.weights.get())
#select_sub_proc_model is required to compile and run hierarchical Processes
run_cfg = Loihi1SimCfg(select_sub_proc_model=True)
denseLayer.compile(run_cfg=run_cfg)


weights: 
 [[3 3 1 1 3 3]
 [3 3 4 1 2 2]]


#### Run Connected DenseLayer _Processes_

In [21]:
dim=(3,3)
#initialize random weight mats
weights0 = np.zeros(shape=dim)
weights0[1,1]=1
weights1 = weights0
#weights0 = np.random.randint(5,size=dim)
#weights1 = np.random.randint(5,size=dim)
#instantiate 2 DenseLayers
layer0 = DenseLayer(shape=dim,weights=weights0, bias=3, vth=10)
layer1 = DenseLayer(shape=dim,weights=weights1, bias=3, vth=10)
#connect layer 0 to layer 1
layer0.s_out.connect(layer1.s_in)
#running layer 1 runs all connected layers (layer 0)
layer1.run(condition=RunSteps(num_steps=1), run_cfg=run_cfg)
print('Layer 1 weights: \n', layer1.weights.get())
print('Layer 0 v: ', layer0.v.get())
print('Layer 1 v: ', layer1.v.get())
layer1.run(condition=RunSteps(num_steps=1), run_cfg=run_cfg)
print('Layer 1 weights: \n', layer1.weights.get())
print('Layer 0 v: ', layer0.v.get())
print('Layer 1 v: ', layer1.v.get())
layer1.run(condition=RunSteps(num_steps=1), run_cfg=run_cfg)
print('Layer 1 weights: \n', layer1.weights.get())
print('Layer 0 v: ', layer0.v.get())
print('Layer 1 v: ', layer1.v.get())
layer1.run(condition=RunSteps(num_steps=1), run_cfg=run_cfg)
print('Layer 1 weights: \n', layer1.weights.get())
print('Layer 0 v: ', layer0.v.get())
print('Layer 1 v: ', layer1.v.get())

Layer 1 weights: 
 [[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Layer 0 v:  [3. 3. 3.]
Layer 1 v:  [3. 3. 3.]
Layer 1 weights: 
 [[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Layer 0 v:  [6. 6. 6.]
Layer 1 v:  [6. 6. 6.]
Layer 1 weights: 
 [[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Layer 0 v:  [9. 9. 9.]
Layer 1 v:  [9. 9. 9.]
Layer 1 weights: 
 [[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Layer 0 v:  [0. 0. 0.]
Layer 1 v:  [0. 0. 0.]


## How to learn more?

If you want to find out more about _SubProcessModels_, have a look at the [Lava documentation](https://lava-nc.org/) or dive into the [source code](https://github.com/intel-nrc-ecosystem/lava-core-rethink/blob/main/lava/magma/core/model/sub/model.py).

To receive regular updates on the latest developments and releases of the Lava Software Framework please subscribe to [our newsletter](http://eepurl.com/hJCyhb).